# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [76]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

# from sklearn.metrics import confusion_matrix
# 
# from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /Users/stingl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stingl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stingl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [77]:
# load data from database
engine = create_engine('sqlite:///messages.db')
connection = engine.connect()
df = pd.read_sql_table("messages", con=connection)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] 

### 2. Write a tokenization function to process your text data

In [78]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [79]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators = 10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [80]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [81]:
Y_pred = pipeline.predict(X_test)

In [82]:
pd.DataFrame(Y_pred).iloc[:, 0]

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       0
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      0
21      0
22      1
23      1
24      0
25      1
26      1
27      1
28      1
29      0
       ..
7779    1
7780    0
7781    0
7782    1
7783    1
7784    1
7785    1
7786    1
7787    1
7788    0
7789    0
7790    1
7791    1
7792    1
7793    0
7794    1
7795    0
7796    1
7797    1
7798    1
7799    1
7800    1
7801    1
7802    1
7803    0
7804    1
7805    1
7806    0
7807    0
7808    1
Name: 0, Length: 7809, dtype: int64

In [83]:
print(classification_report(Y_test.iloc[:, 0], pd.DataFrame(Y_pred).iloc[:, 0]))

              precision    recall  f1-score   support

           0       0.63      0.45      0.52      1868
           1       0.84      0.92      0.88      5941

   micro avg       0.80      0.80      0.80      7809
   macro avg       0.73      0.68      0.70      7809
weighted avg       0.79      0.80      0.79      7809



In [84]:
Y_test.iloc[:, 0].value_counts()

1    5941
0    1868
Name: related, dtype: int64

In [72]:
Y_test.iloc[:, 0]
pd.DataFrame(Y_pred).iloc[:, 0]

0       1
1       1
2       1
3       1
4       1
5       1
6       0
7       1
8       1
9       1
10      0
11      1
12      1
13      0
14      1
15      1
16      1
17      1
18      0
19      1
20      1
21      0
22      1
23      0
24      0
25      1
26      0
27      1
28      1
29      0
       ..
7835    1
7836    1
7837    1
7838    1
7839    1
7840    1
7841    1
7842    1
7843    1
7844    0
7845    1
7846    1
7847    1
7848    1
7849    1
7850    1
7851    1
7852    1
7853    1
7854    1
7855    1
7856    1
7857    1
7858    1
7859    0
7860    1
7861    0
7862    1
7863    1
7864    0
Name: 0, Length: 7865, dtype: int64

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.